# Extract respiratory signals with RAFT

Recurrent All-Pairs Field Transforms (RAFT) is a deep learning model for optical flow estimation. The optical flow directions and magnitudes can be used to extract respiratory signals from videos. This notebook demonstrates how to use RAFT to extract respiratory signals from videos.

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband16'
setting = '101_natural_lighting'

video_path = dataset.get_video_path(subject, setting)

## Load the RAFT model

In [ ]:
import respiration.utils as utils
import respiration.extractor.optical_flow_raft as raft

device = utils.get_torch_device()
model = raft.load_model('raft_small', device)

## Extract optical flow from the video

In [ ]:
import torch
import numpy as np
from tqdm.auto import tqdm

# Number of frames that are processed at once
batch_size = 10

param = utils.get_video_params(video_path)

# Only get the first 12 seconds of the video
param.num_frames = param.fps * 4

# Store the optical flows vectors (N, 2, H, W)
optical_flows = np.zeros((param.num_frames, 2, param.height, param.width), dtype=np.float32)

# Store the frames (N, H, W, 3)
frames = np.zeros((param.num_frames, param.height, param.width, 3), dtype=np.uint8)

# Extract the optical flow from the video in batches
for start in tqdm(range(0, param.num_frames, batch_size)):
    # Calculate the number of frames to process in this batch
    num_frames = min(start + batch_size, param.num_frames) - start

    chunk, _ = utils.read_video_rgb(video_path, num_frames, start)
    frames[start:start + num_frames] = chunk

    chunk = raft.preprocess(chunk, device)

    with torch.no_grad():
        # Split the frames into odd and even frames to calculate optical flow on consecutive frames
        flows = model(chunk[::2], chunk[1::2])

    # Garbage collect...
    del chunk

    # Only keep the last flow iteration
    flows = flows[-1]

    for idx in range(flows.shape[0]):
        # Add the optical flow to the numpy array
        optical_flows[start + idx] = flows[idx].cpu().numpy()

## Visualize the optical flow

In [ ]:
import matplotlib.pyplot as plt

arrow_frame = raft.draw_flow(frames[0], optical_flows[0])
flow_frame = raft.image_from_flow(optical_flows[0])

fig, ax = plt.subplots(1, 2, figsize=(20, 8))
ax[0].imshow(arrow_frame)
ax[0].set_title('Optical flow arrows')
ax[1].imshow(flow_frame)
ax[1].set_title('Optical flow magnitude')

utils.store_figure(fig, 'raft', 'optical_flow')

## Extract the respiratory signal

1. Find the region of interest (ROI) on the chest
2. Calculate the motion magnitude in the ROI
3. Plot the motion magnitude over time

In [ ]:
import respiration.roi as roi

# Find the chest region
x, y, w, h = roi.detect_chest(frames[0])

# Get only the optical flows in the chest region
roi_flows = optical_flows[:, :, y:y + h, x:x + w]

# Calculate motion magnitude by squaring the x and y components and taking the square root
magnitudes = np.sqrt(np.sum(roi_flows ** 2, axis=(1, 2)))

In [ ]:
mean_curve = np.mean(magnitudes, axis=1)
std_curve = np.std(magnitudes, axis=1)

fig, ax = plt.subplots(1, 1, figsize=(20, 8))
ax.plot(mean_curve, label='Mean')
ax.fill_between(
    np.arange(len(mean_curve)),
    mean_curve - std_curve,
    mean_curve + std_curve,
    alpha=0.3,
    label='Standard deviation')
ax.set_title('Motion magnitude in the ROI')
ax.set_xlabel('Frame')
ax.set_ylabel('Motion magnitude')

utils.store_figure(fig, 'raft', 'roi_magnitudes')